In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import sequence

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
vocab_size = 10000
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)

17464789/17464789 [==============================] - 0s 0us/step


In [4]:
word_index = tf.keras.datasets.imdb.get_word_index()

1641221/1641221 [==============================] - 0s 0us/step


In [5]:
word_index = {k:(v+3) for k, v in word_index.items()}

word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNKNOWN>"] = 2
word_index["<UNUSED>"] = 3


reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [6]:
# 這邊可以調
review_length = 500

x_train = sequence.pad_sequences(x_train, maxlen = review_length)
x_test = sequence.pad_sequences(x_test, maxlen = review_length)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(25000, 500)
(25000,)
(25000, 500)
(25000,)


In [7]:
from tensorflow.keras import layers, models


In [8]:
lstm = models.Sequential(
    [
        layers.Embedding(
            input_dim = vocab_size,
            output_dim = 32,
            input_length = review_length,
        ),

        layers.LSTM(
            units = 32,
        ),

        layers.Dense(
            units = 1,
            activation = "sigmoid"
        ),
    ],
    name = "LSTM"
)

lstm.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.binary_crossentropy,
    metrics = ["accuracy"],
)

lstm.summary()

Model: "LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           320000    
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 328353 (1.25 MB)
Trainable params: 328353 (1.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
history = lstm.fit(
    x_train,
    y_train,
    batch_size = 250,
    epochs = 10,
    validation_split = 0.2,
)

Epoch 1/10
80/80 [==============================] - 47s 554ms/step - loss: 0.5905 - accuracy: 0.6723 - val_loss: 0.4126 - val_accuracy: 0.8174
Epoch 2/10
80/80 [==============================] - 41s 515ms/step - loss: 0.3002 - accuracy: 0.8828 - val_loss: 0.3274 - val_accuracy: 0.8784
Epoch 3/10
80/80 [==============================] - 43s 533ms/step - loss: 0.2058 - accuracy: 0.9258 - val_loss: 0.2862 - val_accuracy: 0.8838
Epoch 4/10
80/80 [==============================] - 42s 529ms/step - loss: 0.1514 - accuracy: 0.9495 - val_loss: 0.2932 - val_accuracy: 0.8858
Epoch 5/10
80/80 [==============================] - 42s 528ms/step - loss: 0.1181 - accuracy: 0.9620 - val_loss: 0.3101 - val_accuracy: 0.8836
Epoch 6/10
80/80 [==============================] - 46s 581ms/step - loss: 0.1056 - accuracy: 0.9668 - val_loss: 0.3737 - val_accuracy: 0.8832
Epoch 7/10
80/80 [==============================] - 44s 552ms/step - loss: 0.0851 - accuracy: 0.9746 - val_loss: 0.3884 - val_accuracy: 0.8766

In [10]:
import json

history_dict = history.history

In [11]:
json.dump(history_dict, open("/content/drive/MyDrive/Colab Notebooks/imdb/history/lstm.json", 'w'))

In [12]:
lstm.save("/content/drive/MyDrive/Colab Notebooks/imdb/model/lstm.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
